In [4]:
# !pip install geemap
# !pip install geopandas

In [20]:
import ee
import os
import geemap
import geopandas
import folium

# Trigger the authentication flow
ee.Authenticate()

# Initialize library
ee.Initialize()

In [6]:
def orbit_granuleID(aoi, collection_ID='COPERNICUS/S2_SR', start_date='2021-01-01', end_date='2021-01-10', orbit_pass='DESCENDING'):
    """
    returns orbit number or granule ID for Sentinel-1 or Sentinel-2 for a given area of interest
    
    aoi: area of interest e.g. geojson or shapefile in EPSG:4326
    collection_ID : GEE collection ID e.g. 'COPERNICUS/S2_SR' or 'COPERNICUS/S1_GRD'"
    start_date: collection start date in YYYY-MM-DD format
    end_date: collection end date in YYYY-MM-DD format
    orbit_pass= orbit property metadata for Sentinel-1 only
    
    """

    if aoi.endswith('geojson'):
        aoi = geemap.geojson_to_ee('sample_farm.geojson')
    
    elif aoi.endswith('.shp'):
        aoi = geemap.shp_to_ee('sample_farm.shp')

    else:
        print('invalid input for area of interest')
    
    # get unique granule_ID or orbit number
    if 'S2' in collection_ID: 
        collection = ee.ImageCollection(collection_ID).filterDate(start_date,end_date).filterBounds(aoi).distinct('MGRS_TILE')

        granule = collection.aggregate_array('MGRS_TILE')
        granule_geometries = ee.FeatureCollection(collection.map(lambda img: ee.Feature(img.geometry(), {'Granule ID': img.get('MGRS_TILE')})))


    elif 'S1'  in collection_ID:
        collection = ee.ImageCollection(collection_ID).filter(ee.Filter.eq('instrumentMode', 'IW')).filterDate(
            start_date, end_date).filterBounds(aoi).filter(ee.Filter.eq('orbitProperties_pass', orbit_pass)).filter(
                ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')).filter(
                     ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')).distinct('relativeOrbitNumber_start')


        granule = collection.aggregate_array('relativeOrbitNumber_start')
        granule_geometries = ee.FeatureCollection(collection.map(lambda img: ee.Feature(img.geometry(), {'Orbit Number': img.get('relativeOrbitNumber_start')})))


    print('Number of unique orbit/granule IDs -->', granule.size().getInfo())
    print('Granule IDs -->' , granule.getInfo())

    return aoi.geometry().getInfo(), granule_geometries.getInfo()
    

## Load area of interest

In [7]:
input_aoi = 'sample_farm.shp'

## Map properties

In [9]:
aoi_style = {'fillColor': '#00000000','color': '#000000'}
footprint_style = {'fillColor': '#00000000','color': '#FF0000'}

## Example : Visualise Sentinel-2 footprints

In [18]:
# get aoi and footprint
collection_ID = 'COPERNICUS/S2_SR' # 'COPERNICUS/S1_GRD' or 'COPERNICUS/S2_SR'
aoi, footprint = orbit_granuleID(input_aoi, collection_ID=collection_ID, start_date='2021-01-01', end_date='2021-01-10', orbit_pass='DESCENDING')

# map properties
frame = folium.Figure(width=1050, height=750)
popup_field = ['Orbit Number'] if 'S1' in collection_ID else ['Granule ID']

map = folium.Map().add_to(frame)
folium.GeoJson(aoi, name='aoi', style_function=lambda x:aoi_style).add_to(map)
folium.GeoJson(footprint, name='footprint', style_function=lambda x:footprint_style).add_child(folium.GeoJsonPopup(fields=popup_field)).add_to(map)

folium.LayerControl(collapsed=False).add_to(map)
map.fit_bounds(map.get_bounds(), padding=(30, 30))

map

Number of unique orbit/granule IDs --> 1
Granule IDs --> ['30UVU']


## Example : Visualise Sentinel-1 footprints

In [19]:
# get aoi and footprint
collection_ID = 'COPERNICUS/S1_GRD' # 'COPERNICUS/S1_GRD' or 'COPERNICUS/S2_SR'
aoi, footprint = orbit_granuleID(input_aoi, collection_ID=collection_ID, start_date='2021-01-01', end_date='2021-01-10', orbit_pass='DESCENDING')


# map properties
popup_field = ['Orbit Number'] if 'S1' in collection_ID else ['Granule ID']

frame2 = folium.Figure(width=1050, height=750)
map2 = folium.Map().add_to(frame2)
folium.GeoJson(aoi, name='aoi', style_function=lambda x:aoi_style).add_to(map2)
folium.GeoJson(footprint, name='footprint', style_function=lambda x:footprint_style).add_child(folium.GeoJsonPopup(fields=popup_field)).add_to(map2)

folium.LayerControl(collapsed=False).add_to(map2)
map2.fit_bounds(map2.get_bounds(), padding=(30, 30))

map2

Number of unique orbit/granule IDs --> 2
Granule IDs --> [52, 154]
